# Animal Faces Transfer Learning

### This notebook contains the code to carry out the analysis of the transfer learning capacilities of the pre-trained model from task 1 and the ImageNet pre-trained model.
### First, the pre-trained model from task 1 will be loaded to get the CNN encoder to extract features to be passed along to a Logistic Regression model for classification. An analysis of the performance of the classifier will be made with respect to the features extracted.
### Then the same procedure will be done but for a ResNet model with ImageNet pre-trained weights.

#### Colorectal Cancer Pre-trained Model

#### ImageNet Pre-trained Model

- Get the model from pytorch with a ImageNet preset

- Remove the classification head

- Load the dataset and preprocess it

- Get the Logistic Regression model

- Extract the features

- Pass the features to the training loop for the classifier 

- Plot the results

- Discussion